In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os

def load_json(file_path): # Load JSON Data
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                print("JSON decode error — file is empty or malformed.")
                return {}
    else:
        return {}  # Return empty dict if file doesn't exist


def update_json(data, key, value): # Modify/Add Data
    data[key] = value
    return data

def save_json(file_path, data): # Save JSON Data
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

def phase_key_to_short(phase_key):
    """
    Convert 'VO2 (space group 14)' to 'VO2_14'
    """
    if " (space group " in phase_key and phase_key.endswith(")"):
        name, sg = phase_key.split(" (space group ")
        sg = sg.rstrip(")")
        return f"{name} ({sg})"
    return phase_key

# variable init 
json_file_path = "Data/prompt1/interpretations_llm_v1_llama1.json"
json_file = load_json(json_file_path)

In [ ]:
# checks that all interpretations have the correct number of phases in the LLM response
json_file_path = "Data/prompt3/interpretations_llm_v2_llama1.json"
json_file = load_json(json_file_path)
for sample in json_file:
    for interpret in json_file[sample]:
        if interpret not in ["Synth_Conditions"]:
            if "LLM_interpretation_likelihood_llama" not in json_file[sample][interpret]:
                print(f"{sample} {interpret} No llama interpretation score")
                continue
            gpt_score = json_file[sample][interpret]["LLM_interpretation_likelihood"]
            llama_score = json_file[sample][interpret]["LLM_interpretation_likelihood_llama"]
            phases_llama = list(json_file[sample][interpret]["LLM_phases_likelihood_llama"].keys())
            phases_chat = list(json_file[sample][interpret]["LLM_phases_likelihood"].keys())
            weightfractions = json_file[sample][interpret]["weight_fraction"]
            if len(phases_llama) != len(weightfractions):
                print(f"llama: {sample} {interpret}")
            if len(phases_chat) != len(weightfractions):
                print(f"chat: {sample} {interpret}")
            
            

llama: ARR_39 I_1
llama: TRI_62 I_1
llama: TRI_62 I_2
llama: TRI_62 I_3
llama: TRI_62 I_4
llama: TRI_62 I_5
llama: TRI_87 I_1
